Target Project: SimpleHTR (https://github.com/githubharald/SimpleHTR) <br>
Target Library: Tensorflow

### 1. Method Invocation

In [1]:
import utils.utils as utils
import data.SimpleHTRMaster as data

import libcst as cst
source_tree = cst.parse_module(data.file_string) #using libcst to parse module

saved = []
invocations = []

class getMethodInvocation(cst.CSTVisitor):
    def visit_Attribute(self, node: cst.Attribute) -> None:
        # print("---------------------")
        # print(node.children)

        try:
            if(node.value.value.value.value == "tf"):
                saved.append(node.value.value.value.value + "." + node.value.value.attr.value + "." + node.value.attr.value)
                string = node.value.value.value.value + "." + node.value.value.attr.value + "." + node.value.attr.value + "." + node.attr.value
                invocations.append(string)
                return
        except:
            pass

        try:
            if(node.value.value.value.value == "tf"):
                saved.append(node.value.value.value.value + "." + node.value.value.attr.value + "." + node.value.attr.value)
                string = node.value.value.value.value + "." + node.value.value.attr.value + "." + node.value.attr.value + "." + node.attr.value
                invocations.append(string)
                return
        except:
            pass

        try:
            if(node.value.value.value == "tf"):
                saved.append(node.value.value.value + "." + node.value.attr.value)
                string = node.value.value.value + "." + node.value.attr.value + "." + node.attr.value
                if(string not in saved ):
                    invocations.append(string)
                return
        except:
            pass

        try:
            if(node.value.value == "tf"):
                string = node.value.value + "." +node.attr.value
                if(string not in saved ):
                    invocations.append(string)
        except:
            pass 

source_tree.visit(getMethodInvocation()) #traverse
print(invocations)
import numpy as np
print("Number of invocations: ", len(invocations))
print("Number of distinct invocations: ", len(np.unique(np.array(invocations))))

['tf.compat.v1.disable_eager_execution', 'tf.compat.v1.placeholder', 'tf.bool', 'tf.compat.v1.placeholder', 'tf.float32', 'tf.compat.v1.get_collection', 'tf.compat.v1.GraphKeys', 'tf.control_dependencies', 'tf.compat.v1.train', 'tf.expand_dims', 'tf.Variable', 'tf.random.truncated_normal', 'tf.nn.conv2d', 'tf.compat.v1.layers', 'tf.nn.relu', 'tf.nn.max_pool2d', 'tf.squeeze', 'tf.compat.v1.nn', 'tf.compat.v1.nn', 'tf.compat.v1.nn', 'tf.expand_dims', 'tf.concat', 'tf.Variable', 'tf.random.truncated_normal', 'tf.squeeze', 'tf.nn.atrous_conv2d', 'tf.transpose', 'tf.SparseTensor', 'tf.compat.v1.placeholder', 'tf.int64', 'tf.compat.v1.placeholder', 'tf.int32', 'tf.compat.v1.placeholder', 'tf.int64', 'tf.compat.v1.placeholder', 'tf.int32', 'tf.reduce_mean', 'tf.compat.v1.nn', 'tf.compat.v1.placeholder', 'tf.float32', 'tf.compat.v1.nn', 'tf.nn.ctc_greedy_decoder', 'tf.nn.ctc_beam_search_decoder', 'tf.nn.softmax', 'tf.compat.v1.Session', 'tf.compat.v1.train', 'tf.__version__', 'tf.compat.v1.Ses

### 2. Call Sequences

In [2]:
statements = source_tree.children
simple_statement_lines = [] #list containing nodes of type SimpleStatementLines
function_def_lines = [] #list containing nodes of type FunctionDef

results = utils.sort_statements(statements)
simple_statement_lines = results[0]
function_def_lines = results[1]
class_def_lines = results[2]

calls = []
calls_pIndex = []

def call_search_SSL(parent, index):
    if(isinstance(parent, cst.Call)):
        try:
            if(parent.func.value.value == 'tf'):
                calls.append(parent)
                calls_pIndex.append(index)
            if(parent.func.value.value.value == 'tf'):
                calls.append(parent)
                calls_pIndex.append(index)
            if(parent.func.value.value.value.value == 'tf'):
                calls.append(parent)
                calls_pIndex.append(index)
        except:
            pass
    if(len(parent.children) == 0):
        return
    else:
        for child in parent.children:
            call_search_SSL(child, index)

for i in range(len(simple_statement_lines)):
    if(isinstance(simple_statement_lines[i], cst.SimpleStatementLine)):
        call_search_SSL(simple_statement_lines[i], i)


In [3]:
simpleStatementLine_calls = []

for i in range(len(calls)):
    simpleStatementLine_calls.append([calls_pIndex[i], calls[i]])

groupings = utils.cluster(calls_pIndex, maxgap=1)

#only 1 function call for tensorflow library in main source tree
#no other function definition in main source tree
#hence check for other class definition trees 

#Call Sequences

call_sequences = []

for group in groupings:
    temp_call_sequence = []
    for called in simpleStatementLine_calls:
        if(called[0] in group):
            #print(called[1])
            temp_call_sequence.append(called[1])
    call_sequences.append(temp_call_sequence)

In [4]:
#only 2nd class definition tree contain tensorflow library API calls
#hence only work on 2nd class definition tree 'class_def_lines[1]'
#this tree does not contain any simplestatementlines that contains tensorflow library API calls
#hence only work on function definitions

results2 = utils.sort_statements(class_def_lines[1].body.body)
results2[1]

groups = []

def call_search_FD(parent):
    if(isinstance(parent, cst.Call)):
        try:
            if(parent.func.value.value == 'tf'):
                group.append(parent)
            if(parent.func.value.value.value == 'tf'):
                group.append(parent)
            if(parent.func.value.value.value.value == 'tf'):
                group.append(parent)
            if(parent.func.value.value.value.value.value == 'tf'):
                group.append(parent)
        except:
            pass
    if(len(parent.children) == 0):
        return
    else:
        for child in parent.children:
            call_search_FD(child)

for function_def in results2[1]:
    if(isinstance(function_def, cst.FunctionDef)):
        group = []
        call_search_FD(function_def)
        groups.append(group)

for group in groups:
    if(len(group) >= 1):
        call_sequences.append(group)

call_sequences

simple_htr_master_call_sequences = call_sequences

%store simple_htr_master_call_sequences


Stored 'simple_htr_master_call_sequences' (list)
